In [1]:
import pandas as pd;
import numpy as np;

def PrintMatrix(x, y, A):    
    AT = np.array(A).T.tolist();
    M = [];
    y = "*" + y;    
    for j in range(len(y)):
        M.append([y[j]] + AT[j]);
    df = pd.DataFrame(M, columns = ['*'] + ['*'] + list(x));    
    print (df.to_string(index=False, col_space=10, justify="left"))

class ScoreParam:
    """Stores the parameters for an alignment scoring function"""
    def __init__(self, match, mismatch, gap):
        self.gap = gap
        self.match = match
        self.mismatch = mismatch

    def matchchar(self, a,b):
        """Return the score for aligning character a with b"""
        assert len(a) == len(b) == 1
        if a==b:
            return self.match
        else:
            return self.mismatch

    def __str__(self):
        return "match = %d; mismatch = %d; gap = %d" % (
                self.match, self.mismatch, self.gap
        )

def GlobalAlign(x, y, score=ScoreParam(2, 0, -1), show_steps=0):
    M = [[0]*(len(y) + 1) for i in range(len(x) + 1)]

    for i in range(1, len(x)+1):
        M[i][0] = i * score.gap;

    for i in range(1, len(y)+1):
        M[0][i] = i * score.gap;
        
    for j in range(1, len(y)+1):        
        for i in range(1, len(x)+1):        
            M[i][j] = max(score.matchchar(x[i-1], y[j-1]) + M[i-1][j-1],
                    score.gap + M[i][j-1],
                    score.gap + M[i-1][j]
            )  
        if (show_steps):
            print ("loop " + str(j) + ":")
            print_matrix(x,y,M)
            print ("")
    
    print ("x = %s" %(x))
    print ("y = %s" %(y))
    print ("Scoring Parameters: %s" % (score))
    print ("\nScore Matrix: ")
    PrintMatrix(x,y,M)
    TraceMatrix(x, y, M, show_steps)

def TraceMatrix(x, y, M, show_steps):
    xpos = len(x);    
    ypos = len(y);
    x_aligned = "";
    y_aligned = "";
    aligned = "";
    
    #init alignment
    x_aligned = x[xpos-1];
    if (x[xpos-1] == y[ypos-1]):
        y_aligned = y[ypos-1];
    else:
        y_aligned = "_";
    xpos += -1;
    ypos += -1;    
    
    x_aligned, y_aligned, aligned = GenerateAlignment(x, y, xpos, ypos, M, x_aligned, y_aligned, aligned, show_steps);
    
    print ("\nSequence Alignment:")
    print (x_aligned[::-1])
    print (aligned[::-1])
    print (y_aligned[::-1])    

def GenerateAlignment(x, y, xpos, ypos, M, x_aligned, y_aligned, aligned, show_steps):
    if (xpos>0 or ypos>0):
        dirDict = {'left':M[xpos-1][ypos], 'diag': M[xpos-1][ypos-1], 'up': M[xpos][ypos-1]};    
        dirM = [key for key, value in dirDict.items() if value == max(dirDict.values())];       

        if (dirM[0] == "up"):
            y_aligned += y[ypos-1];        
            x_aligned += "_";
            ypos += -1;            
        elif(dirM[0] == "left"):
            x_aligned += x[xpos-1];        
            y_aligned += "_";
            xpos += -1;            
        else: #diag
            y_aligned += y[ypos-1];        
            x_aligned += x[xpos-1];        
            xpos += -1;
            ypos += -1;
        # Alignment indicator
        if (x_aligned[len(x_aligned)-1] ==  y_aligned[len(y_aligned)-1]):
            aligned += "|";
        else:
            aligned += " ";

        if (show_steps):
            print ("")
            print (y_aligned[::-1])
            print (aligned[::-1])
            print (x_aligned[::-1])
        return GenerateAlignment(x, y, xpos, ypos, M, x_aligned, y_aligned, aligned, show_steps);
    else:
        return x_aligned, y_aligned, aligned;
        
GlobalAlign("CACG", "CATTG", ScoreParam(match=2, mismatch=0, gap=-1), show_steps=0)  

x = CACG
y = CATTG
Scoring Parameters: match = 2; mismatch = 0; gap = -1

Score Matrix: 
*          *          C          A          C          G        
*          0         -1         -2         -3         -4        
C         -1          2          1          0         -1        
A         -2          1          4          3          2        
T         -3          0          3          4          3        
T         -4         -1          2          3          4        
G         -5         -2          1          2          5

Sequence Alignment:
CAC_G
||  
CATTG
